In [35]:
from skimage import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from sklearn.ensemble import IsolationForest
import cv2
import os

In [36]:
#configuracion necesaria de pyplot para ver las imagenes en escala de grises
plt.rcParams['image.cmap'] = 'gray'

# comando de Jupyter para que las imagenes se muestren automaticamente 
%matplotlib inline

In [37]:
def pixRelevant(raw, mask):
    """
    Función que devuelve las áreas relevantes para el análisis mediante aplicar una máscara
    """
    #Convertimos todos los pixelex de la máscara en 0 o 1
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i][j] != 0:
              mask[i][j] = 1
    relevant_img = raw * mask
    return relevant_img

In [38]:
def numLabels(img_matrix):
    """
    Función que devuelve el número de etiquetas que usó el experto para 
    catalogar las diferentes zonas en el pulmón, se cuenta la zona 
    negra como una etiqueta.
    """
    labels_exp = []
    for i in range(img_matrix.shape[0]):
        for j in range(img_matrix.shape[1]):
            if img_matrix[i][j]  not in labels_exp:
                labels_exp.append(img_matrix[i][j])
    return len(labels_exp)

In [39]:
def createAttrImg(img, 
                  kernel_mean = 10, 
                  kernel_gauss = 0, 
                  kernel_median = 9, 
                  canny_min = 100, 
                  canny_max = 500, 
                  d_bilateral = 10,
                  sigmaColor_bilateral = 5, 
                  sigmaSpace_bilateral = 5):

    """
    Función que devuelve un objeto de tipo DataFrame, este contiene como filas a cada uno de los pixeles de la imagen.
    cada atributo representa un filtro aplicado en la imagen.
    """
    #Sin filtro
    img_inline = np.reshape(img, (1, img.shape[0]*img.shape[1])).squeeze()
    df_img_res = pd.DataFrame(img_inline, columns=['Raw'])


    #Filtro Medio
    image_test = cv2.blur(img, (kernel_mean, kernel_mean))
    df_img_res['Mean'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()

    #Filtro Gaussiano
    image_test = cv2.GaussianBlur(img, (kernel_gauss, kernel_gauss), cv2.BORDER_DEFAULT)
    df_img_res['Gauss'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()

    #Filtro Mediano
    image_test = cv2.medianBlur(img, kernel_median)
    df_img_res['Median'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()

    #Filtro Conny
    image_test = cv2.Canny(img, canny_min, canny_max)
    df_img_res['Conny'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()

    #Filtro Horizontal
    kernel = np.array([[-1.0, -1.0, -1.0], 
                   [2.0, 2.0, 2.0],
                   [-1.0, -1.0, -1.0]])
    image_test = cv2.filter2D(img, -1, kernel)
    df_img_res['Horizontal'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()

    #Filtro Vertical
    kernel = np.array([[-1.0, 2.0, -1.0], 
                   [-1.0, 2.0, -1.0],
                   [-1.0, 2.0, -1.0]])
    image_test = cv2.filter2D(img, -1, kernel)
    df_img_res['Vertical'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()

    #Filtro Laplaciano
    kernel = np.array([[0.0, -1.0, 0.0], 
                   [-1.0, 4.0, -1.0],
                   [0.0, -1.0, 0.0]])
    image_test = cv2.filter2D(img,-1, kernel)
    df_img_res['Laplacian'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()

    #Filtro Bilateral
    image_test = cv2.bilateralFilter(img, d_bilateral, sigmaColor_bilateral, sigmaSpace_bilateral, cv2.BORDER_DEFAULT)
    df_img_res['Bilateral'] = np.reshape(image_test, (1, img.shape[0]*img.shape[1])).squeeze()
    
    return df_img_res

In [40]:
def labels_iForest(attr, shape_img_output, grade):
    """
    La función aplicaráa Isolation Forest a un conjunto de datos de atributos de una imagen, 
    posteriormente etiquetará cada vector dependiendo del grado que sea ingresado.
    Entrada:
        attr: Conjunto de datos con atributos de una imagen.
        shape_img_output: Especifica las dimensiones de salida de la imagen.
        grade: núúmero de etiquetas que se aplicarán al conjunto.
    Salida:
        labels: Etiquetas para cada vector.
    """
    # Se selecciona la cantidad de árboles y máximo tamaño de los árboles,
    # según la publicación original.
    clf = IsolationForest(n_estimators=100, max_samples=256)
    clf.fit(attr)
    outlier_score_img = np.floor(-grade * clf.score_samples(attr))
    return np.reshape(outlier_score_img, shape_img_output)

# Obtención de los atributos para cada imagen.

In [9]:
for i in range(1, 11):
    train_tmp  = io.imread("/content/drive/MyDrive/comia2021/Train/tr_im{}.png".format(i))
    lung_mask_tmp = io.imread("/content/drive/MyDrive/comia2021/LungMask/tr_lungmask{}.png".format(i))
    train_lung_mask_tmp = pixRelevant(train_tmp, lung_mask_tmp)
    df_tmp = createAttrImg(train_lung_mask_tmp)
    df_tmp.to_csv("/content/drive/MyDrive/comia2021/Attr/attr_img{}.csv".format(i), sep='\t', index=None)
    

#Segmentación por medio de Isolation Forest

Seleccionamos la lista de conjuntos de datos que se crearon anteriormente.

Extraemos la lista de imágenes que contienen las máscaras creadas por el experto.

In [10]:
for i in range(1, 11):
    mask_exp_tmp = io.imread("/content/drive/MyDrive/comia2021/Mask/tr_mask{}.png".format(i))
    labels_mask_tmp = numLabels(mask_exp_tmp)
    df_tmp = pd.read_csv("/content/drive/MyDrive/comia2021/Attr/attr_img{}.csv".format(i), sep='\t')
    labels_tmp = labels_iForest(df_tmp, (512, 512), labels_mask_tmp)
    df_out = pd.DataFrame(np.reshape(labels_tmp, (262144, 1)), columns=(['Label']), index=None)
    df_out.to_csv("/content/drive/MyDrive/comia2021/Result_Data/res_data{}.csv".format(i), sep='\t')
    io.imsave("/content/drive/MyDrive/comia2021/Result_Image/res_img{}.png".format(i), labels_tmp)

#Cálculo del error entre imágenes

In [55]:
for i in range(1, 11):
    expert_img = io.imread("/content/drive/MyDrive/comia2021/Mask/tr_mask{}.png".format(i))
    result_img =io.imread("/content/drive/MyDrive/comia2021/Result_Image/res_img{}.png".format(i))
    mean_row = 0
    mean_col = 0
    for row in range(expert_img.shape[0]):
        for col in range(expert_img.shape[1]):
            mean_row += np.abs(expert_img[row][col]-result_img[row][col])
        mean_col += mean_row / expert_img.shape[0]
        mean_row = 0
    mean_col /= expert_img.shape[1]
    print("Error entre tr_mask{}.png y res_img{}.png es de {}.".format(i, i, mean_col))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in ubyte_scalars
  


Error entre tr_mask1.png y res_img1.png es de 21.40704345703125.
Error entre tr_mask2.png y res_img2.png es de 37.92649841308594.
Error entre tr_mask3.png y res_img3.png es de 0.11958694458007812.
Error entre tr_mask4.png y res_img4.png es de 17.673057556152344.
Error entre tr_mask5.png y res_img5.png es de 2.583099365234375.
Error entre tr_mask6.png y res_img6.png es de 1.6251335144042969.
Error entre tr_mask7.png y res_img7.png es de 15.4342041015625.
Error entre tr_mask8.png y res_img8.png es de 33.75940704345703.
Error entre tr_mask9.png y res_img9.png es de 40.186824798583984.
Error entre tr_mask10.png y res_img10.png es de 12.627769470214844.
